In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")


span = int(input("Enter number of spans : "))
length = int(input("Enter length value : "))
height = int(input("Enter height value : "))

def construct_ncm(span):
    # construct an empty array
    arr = np.zeros([(3*span)+(span-1), 3])
    
    # construct dataframe from arr
    ncm = pd.DataFrame(arr, columns=['x','y','z'])
    x = []
    for i in range((3*span)+(span-1)):
        x.append(i+1)
    
    # construct first column
    ncm['x'] = x
    ncm = ncm.set_index('x')
    
    # construnct ncm 
    i=1
    j=1
    l= span+2
    k=1

    for index, row in ncm.iterrows():
        # logic for first few columns in ncm where index is <= span
        if(index<=span):
            row['y'] = i
            row['z'] = i+1
            i+=1
        # logic for columns in ncm where span < index <= 3* span
        elif(index>span and index<= 3*span):
            if(index == span+1):
                row['y'] = j
                row['z'] = j + span +1
            elif(index == 3*span):
                row['y'] = span +1
                row['z'] = row['y'] + span
            else:
                if(k==1):
                    j+=1
                    row['y']=j
                    row['z']= j+span
                    k+=1
                elif(k==2):
                    row['y']= j
                    row['z'] = j+span+1
                    k=1
        # logic for remaining columns
        else:
            row['y'] = l
            row['z'] = l+1
            l+=1
            
    # convert float to int        
    ncm['y'] = ncm['y'].astype('int')
    ncm['z'] = ncm['z'].astype('int')
    ncm = ncm.reset_index()
    return ncm

ncm_matrix = construct_ncm(span)

print('NCM matrix :')
print(ncm_matrix.values)


# construct ECM matrix

def construct_ecm(ncm):
    # reset the index
    ncm1 = ncm.set_index('x')
    
    # iterate over the ncm
    a=[]
    for index,row in ncm1.iterrows():
        b=[]
        b.append(index)
        c = ncm1.loc[(ncm1['y'].isin([row['y'],row['z']])) | (ncm1['z'].isin([row['y'],row['z']]))].index.tolist()
        c.remove(index)
        c=list(set(c))
        b.extend(c)
        a.append(b)
    
    t = pd.DataFrame(a).fillna(0)
    # change the dtype to int
    for col in t:
        t[col] = t[col].astype('int')
        
    return t


ecm = construct_ecm(ncm_matrix)
print("ECM matrix : ")
print(ecm.values)


# construct CM matrix
def construct_cm(ncm):
    # define node position value matrix
    arr = pd.DataFrame(np.zeros([(2*span)+1, 2], dtype='int'), columns={'x','y'})
    
    arr['node'] = arr.index +1
    arr = arr[['node','x','y']]
    
    # construct node postion value matrix based on length & height
    i=0
    j = 0.5
    for index, row in arr.iterrows():
        if (index <= span):
                row['x'] = i*length
                i+=1
        else:
            row['x'] = j*length
            row['y'] = height
            j+=1
    # construct cm matrix
    cm = ncm.copy()
    cm['x1'] = 0
    cm['y1'] = 0
    cm['x2'] = 0
    cm['y2'] = 0
    cm = cm[['x','y','x1','y1', 'z', 'x2', 'y2']]   
     
    for index,row in cm.iterrows():
        row['x1']=arr.loc[arr['node']==row['y'],'x'].values
        row['y1']=arr.loc[arr['node']==row['y'],'y'].values
        
        row['x2']=arr.loc[arr['node']==row['z'],'x'].values
        row['y2']=arr.loc[arr['node']==row['z'],'y'].values  
    
    return cm
    
cm = construct_cm(ncm_matrix)
print("CM matrix : ")
print(cm.values)

# plot the truss
df1 = cm[['x','x1','y1']]
df2 = cm[['y','x2','y2']].rename(columns={'y':'x', 'x2':'x1','y2':'y1'})
df3 = pd.concat([df1,df2], axis=0)
df4 = df3.sort_values(by=['x1', 'y1'], ascending=True)

# actual plotting
df5 = df4.loc[df4['y1']==0]
df6 = df4.loc[df4['y1']==height]

fig = plt.figure(figsize=(8,5), dpi=100)
plt.plot('x1', 'y1', data=df4, marker='*', color='g')
plt.plot('x1', 'y1', data=df5, color='g')
plt.plot('x1', 'y1', data=df6, color='g')
plt.title("Truss")
plt.savefig("truss.png")

print("Done with plotting truss with specified input. Pls check truss.png")





Enter number of spans : 2
Enter length value : 3
Enter height value : 3
NCM matrix :
[[1 1 2]
 [2 2 3]
 [3 1 4]
 [4 2 4]
 [5 2 5]
 [6 3 5]
 [7 4 5]]
ECM matrix : 
[[1 2 3 4 5 0]
 [2 1 4 5 6 0]
 [3 1 4 7 0 0]
 [4 1 2 3 5 7]
 [5 1 2 4 6 7]
 [6 2 5 7 0 0]
 [7 3 4 5 6 0]]
CM matrix : 
[[1 1 0 0 2 3 0]
 [2 2 3 0 3 6 0]
 [3 1 0 0 4 1 3]
 [4 2 3 0 4 1 3]
 [5 2 3 0 5 4 3]
 [6 3 6 0 5 4 3]
 [7 4 1 3 5 4 3]]
Done with plotting truss with specified input. Pls check truss.png
